In [1]:
import pandas as pd
import numpy as np
import talib as talib
from pyecharts import options as opts
from pyecharts.globals import SymbolType
from pyecharts.charts import *
# from pyecharts.

from datetime import datetime,timedelta
import matplotlib.pyplot as plt
# %matplotlib inline
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

#使用tushare获取交易数据
#设置token
import tushare as ts
#注意token更换为你在tushare网站上获取的
token='36d50a0386e9066b5c79c2de58ea2d3a9b980020041adc910f76bfc7'
ts.set_token(token)
pro=ts.pro_api()
index={'上证综指': '000001.SH',
        '深证成指': '399001.SZ',
        '沪深300': '000300.SH',
        '创业板指': '399006.SZ',
        '上证50': '000016.SH',
        '中证500': '000905.SH',
        '中小板指': '399005.SZ',
        '上证180': '000010.SH'}
#获取当前交易的股票代码和名称
def get_code():
    df = pro.stock_basic(exchange='', list_status='L')
    codes=df.ts_code.values
    names=df.name.values
    stock=dict(zip(names,codes))
    #合并指数和个股成一个字典
    stocks=dict(stock,**index)
    return stocks

#获取行情数据
def get_daily_data(stock,start,end):
    #如果代码在字典index里，则取的是指数数据
    code=get_code()[stock]
    if code in index.values():
        df=pro.index_daily(ts_code=code,start_date=start, end_date=end)
    #否则取的是个股数据
    else:
        #df=pro.daily(ts_code=code, adj='qfq',start_date=start, end_date=end)
        df=ts.pro_bar(ts_code=code, adj='qfq', start_date=start, end_date=end)
    #将交易日期设置为索引值
    df.index=pd.to_datetime(df.trade_date)
    df=df.sort_index()
    #计算收益率
    df['ret']=df.close/df.close.shift(1)-1
    return df

def get_line(xdatas, data_dic):
    line= Line()
    line.add_xaxis(xaxis_data=xdatas)
    for key in data_dic:
        value = data_dic[key];
        line.add_yaxis(
            series_name=value['name'],
            y_axis=value['datas'],
            symbol="none",
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.9),
            label_opts=opts.LabelOpts(is_show=False),
        )
    line.set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
    return line

def get_kline (xdatas,kdatas,stock_name):
    c = (
        Kline()
        .add_xaxis(xaxis_data=xdatas)
        .add_yaxis(
            series_name='kline',
            y_axis=kdatas
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                    title=stock_name,
                    subtitle="MA",
            ),
            xaxis_opts=opts.AxisOpts(
                        is_scale=True,
                        type_="category",
                        splitarea_opts=opts.SplitAreaOpts(
                        is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                    ),
            ),
            yaxis_opts=opts.AxisOpts(
                    is_scale=True,
                    splitarea_opts=opts.SplitAreaOpts(
                        is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                    ),
            ),
            legend_opts=opts.LegendOpts(
                is_show=False, pos_bottom=10, pos_left="center"
            ),
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0,1],
                    type_="slider",
                    pos_top="90%",
                    range_start=0,
                    range_end=100,
                ),
#                 opts.DataZoomOpts(
#                     is_show=True,
#                     yaxis_index=[0],
#                     type_="slider",
#                     orient = 'vertical',
# #                     pos_top="90%",
#                     pos_right="0%",
#                     range_start=0,
#                     range_end=100,
#                 ),
            ],
            tooltip_opts=opts.TooltipOpts(
                    trigger="axis",
                    axis_pointer_type="cross",
                    background_color="rgba(245, 245, 245, 0.8)",
                    border_width=1,
                    border_color="#ccc",
                    textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
            visualmap_opts=opts.VisualMapOpts(
                    is_show=True,
                    dimension=2,
                    series_index=9,
                    is_piecewise=True,
                    pieces=[
                        {"value": 1, "color": "#ec0000"},
                        {"value": -1, "color": "#00da3c"},
                    ],
                ),
                axispointer_opts=opts.AxisPointerOpts(
                    is_show=True,
                    link=[{"xAxisIndex": "all"}],
                    label=opts.LabelOpts(background_color="#777"),
                ),
                brush_opts=opts.BrushOpts(
                    x_axis_index="all",
                    brush_link="all",
                    out_of_brush={"colorAlpha": 0.1},
                    brush_type="lineX",
                ),
        )
    )
    return c


def get_volome(xdatas,vols,kdatas): 
    bar = (
        Bar()
        .add_xaxis(xaxis_data=xdatas)
        .add_yaxis(
            series_name="Volume",
            yaxis_data=[
                [i, vols[i], 1 if kdatas[i][0] > kdatas[i][1] else -1]
                for i in range(len(kdatas))
            ],
    #         yaxis_data = vols,
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False)
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category",
                is_scale=True,
                grid_index=1,
                boundary_gap=False,
                axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False),
                split_number=20,
                min_="dataMin",
                max_="dataMax",
            ),
            yaxis_opts=opts.AxisOpts(
                grid_index=1,
                is_scale=True,
                split_number=2,
                axislabel_opts=opts.LabelOpts(is_show=False),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )
    return bar

def my_strategy(data):
    x1=data.close>data.up
    x2=data.close.shift(1)<data.up.shift(1)
    x=x1&x2
    y1=data.close<data.down
    y2=data.close.shift(1)>data.down.shift(1)
    y=y1&y2
    data.loc[x,'signal']='buy'
    data.loc[y,'signal']='sell'
    buy_date=(data[data.signal=='buy'].index).strftime('%Y%m%d')
    sell_date=(data[data.signal=='sell'].index).strftime('%Y%m%d')
    buy_close=data[data.signal=='buy'].close.round(2).tolist()
    sell_close=data[data.signal=='sell'].close.round(2).tolist()
    return (buy_date,buy_close,sell_date,sell_close)

def get_grid(stock_name,xdatas,kdatas,linetec_dic,vols):
    kline = get_kline(xdatas, kdatas,stock_name)
    line = get_line(xdatas, linetec_dic)
    bar = get_volome(xdatas, vols, kdatas)
    overlap_kline_line = kline.overlap(line)
    grid_chart = Grid(init_opts=opts.InitOpts(width="900px",height="800px"))
    grid_chart.add(
        overlap_kline_line,
        grid_opts=opts.GridOpts(pos_left="6%", pos_right="8%", height="60%"),
    )
    
    grid_chart.add(
        bar,
        grid_opts=opts.GridOpts(
            pos_left="6%", pos_right="8%", pos_top="70%", height="10%"
        ),
    )
    
    return grid_chart

/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [5]:
sma_periods = [8,17,25,99]
stock_name = '平安银行'
hs=get_daily_data(stock_name,'20100101','')[['trade_date','close','open','high','low','vol']]
v1=np.array(hs.loc[:,['open','close','low','high']])
trade_dates = np.array(hs.trade_date)
vols = np.array(hs['vol'])

#最近N1个交易日最高价
hs['up']=talib.MAX(hs.high,timeperiod=20).shift(1)
#最近N2个交易日最低价
hs['down']=talib.MIN(hs.low,timeperiod=10).shift(1)
#每日真实波动幅度
hs['ATR']=talib.ATR(hs.high,hs.low,hs.close,timeperiod=20)
tul_dic={}
tul_dic['upLine'] = {
    'name':'上轨道线',
    'datas': np.array(hs['up']).round(1).tolist()
}
tul_dic['downLine'] = {
    'name':'下轨道线',
    'datas': np.array(hs['down']).round(1).tolist()    
}

sma_dic= {}
for period in sma_periods:
    name = "SMA" + str(period)
    sma = talib.SMA(np.array(hs.close),period)
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] = sma.round(2).tolist();

bd,bc,sd,sc=my_strategy(hs)
bd

/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


Index(['20100302', '20110303', '20110406', '20110413', '20110704', '20110715',
       '20111028', '20120120', '20120203', '20120228', '20120424', '20120814',
       '20121102', '20121207', '20121214', '20121228', '20130114', '20130118',
       '20130128', '20130201', '20130228', '20130305', '20130520', '20130528',
       '20130813', '20130909', '20131025', '20131029', '20140211', '20140408',
       '20140610', '20140613', '20140724', '20140728', '20141030', '20141128',
       '20141202', '20141204', '20150312', '20150407', '20150410', '20150608',
       '20151105', '20151109', '20160304', '20160318', '20160622', '20160704',
       '20160712', '20160726', '20160729', '20160812', '20161024', '20161122',
       '20161125', '20170124', '20170126', '20170213', '20170215', '20170220',
       '20170525', '20170531', '20170628', '20170710', '20170713', '20170828',
       '20171018', '20171107', '20171113', '20171120', '20180115', '20180824',
       '20180921', '20180926', '20180928', '20190116

In [6]:
kline=get_kline(trade_dates.tolist(), v1.tolist(),stock_name)
# kline.render_notebook();

# lineTul = getline()

esbuy = (
    EffectScatter()
    .add_xaxis(np.array(bd).tolist())
    .add_yaxis("buy", bc)
    .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
#     .set_global_opts(title_opts=opts.TitleOpts(title="EffectScatter-基本示例"))
)
kline.overlap(esbuy)

esSell = (
    EffectScatter()
    .add_xaxis(np.array(sd).tolist())
    .add_yaxis(series_name="sell",y_axis=sc,symbol=SymbolType.TRIANGLE)
    .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
#     .set_global_opts(title_opts=opts.TitleOpts(title="EffectScatter-基本示例"))
)


kline.overlap(esSell)

line = get_line(trade_dates.tolist(), sma_dic)
overlap_kline_line = kline.overlap(line)
line = get_line(trade_dates.tolist(), tul_dic)
overlap_kline_line = kline.overlap(line)


bar = get_volome(trade_dates.tolist(), vols.tolist(), v1.tolist())

grid_chart = Grid(init_opts=opts.InitOpts(width="900px",height="800px"))
grid_chart.add(
    kline,
    grid_opts=opts.GridOpts(pos_left="6%", pos_right="8%", height="60%"),
)
    
grid_chart.add(
    bar,
    grid_opts=opts.GridOpts(
    pos_left="6%", pos_right="8%", pos_top="70%", height="10%"
    ),
)

tab = Tab()
tab.add(grid_chart,'海龟图')
    
tab.render_notebook()

In [7]:
hs2.loc['2016-01-22':'2016-01-22'].index[0]

NameError: name 'hs2' is not defined

In [8]:
import datetime
import time
date=hs2['trade_date']['2016-01-22']

NameError: name 'hs2' is not defined

In [ ]:
dt=datetime.datetime.strptime(date,'%Y%m%d')
dt

In [ ]:
time.mktime(dt.timetuple())

In [ ]:
dtnxt = dt + timedelta(days=3)
dtnxt
hs2['vol','close'][0]


In [ ]:
hs2 = hs2.reset_index(drop=True)


In [ ]:
hs2

In [ ]:
hs2.set_index('trade_date',drop=False)